In [1]:
import numpy as np
from datasets import load_dataset
from datasets import Dataset, DatasetDict, IterableDataset, IterableDatasetDict
import torch
import torch.nn as nn
from transformers import BertTokenizer,DataCollatorWithPadding, TrainingArguments
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import BertForSequenceClassification,BertModel

In [2]:
train_datasets = load_dataset("json", data_files=["./GeoEAG_data/train.json"])
test_datasets = load_dataset("json", data_files=["./GeoEAG_data/val.json"])

Using custom data configuration default-c4134806cc436348
Reusing dataset json (C:\Users\pc\.cache\huggingface\datasets\json\default-c4134806cc436348\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-259030f4700e3c02
Reusing dataset json (C:\Users\pc\.cache\huggingface\datasets\json\default-259030f4700e3c02\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
for p in iter(train_datasets['train']):
    print(p)
    break


train_datasets = train_datasets['train']
test_datasets = test_datasets['train']

{'sentence1': '思明区中华街道大同路思东商厦3梯', 'sentence2': '霞溪路187号霞溪路181-187思东商厦-四梯', 'label': 'partial_match'}


In [4]:
set(test_datasets['label'])
train_datasets

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 96423
})

In [5]:
label2id = {
    'not_match': 0,
    'partial_match': 1,
    'exact_match' : 2,
}
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [6]:

def map_labels(example):
    example['label'] = label2id[example['label']]
    return example
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding=True, max_length= 256)
 

train_datasets = train_datasets.map(map_labels)
train_datasets = train_datasets.map(tokenize_function, batched=True)

test_datasets = test_datasets.map(map_labels)
test_datasets = test_datasets.map(tokenize_function, batched=True)

dataset_all = DatasetDict({"train": train_datasets, "valid": test_datasets})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)





Loading cached processed dataset at C:\Users\pc\.cache\huggingface\datasets\json\default-c4134806cc436348\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-31a8345c3a547760.arrow
Loading cached processed dataset at C:\Users\pc\.cache\huggingface\datasets\json\default-c4134806cc436348\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-376b5901e9de7a67.arrow
Loading cached processed dataset at C:\Users\pc\.cache\huggingface\datasets\json\default-259030f4700e3c02\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-19e0a5468af475e5.arrow
Loading cached processed dataset at C:\Users\pc\.cache\huggingface\datasets\json\default-259030f4700e3c02\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-23cffb125c81fed7.arrow


In [7]:

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
 
trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset_all["train"],
    eval_dataset=dataset_all["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
trainer.train() #单卡训练慢
# max_len = len(dataset_all['train']['label'])
# max_len
# dataset_all['train']['label']# 0:58525 1:31891 2:6007
# for i,data in enumerate(dataset_all['train']['input_ids']):
#     # print(len(data))
#     # break
#     # if len(data)!=81:
#     print(i,len(data))

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2. If sentence1, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 96423
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18081


  0%|          | 0/18081 [00:00<?, ?it/s]

Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json


{'loss': 0.652, 'learning_rate': 4.861456777833085e-05, 'epoch': 0.08}


Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
Saving model checkpoint to test-trainer\checkpoint-1000
Configuration saved in test-trainer\checkpoint-1000\config.json


...

Model weights saved in test-trainer\checkpoint-18000\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-18000\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-18000\special_tokens_map.json
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\SoftInstall\Anaconda3\envs\pytorch\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2. If sentence1, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore th

  0%|          | 0/1436 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.5201207995414734, 'eval_runtime': 123.419, 'eval_samples_per_second': 186.146, 'eval_steps_per_second': 11.635, 'epoch': 3.0}
{'train_runtime': 6012.0692, 'train_samples_per_second': 48.115, 'train_steps_per_second': 3.007, 'train_loss': 0.4203537940504303, 'epoch': 3.0}


TrainOutput(global_step=18081, training_loss=0.4203537940504303, metrics={'train_runtime': 6012.0692, 'train_samples_per_second': 48.115, 'train_steps_per_second': 3.007, 'train_loss': 0.4203537940504303, 'epoch': 3.0})

In [10]:
predictions = trainer.predict(dataset_all["valid"])
print(predictions.predictions.shape) 

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, sentence2. If sentence1, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 22974
  Batch size = 16


  0%|          | 0/1436 [00:00<?, ?it/s]

(22974, 3)


In [17]:
print(predictions.label_ids.shape) # array([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...], dtype=int64)
print(predictions.metrics)
#pre = np.argmax(predictions.predictions, axis = 2)
print(predictions.predictions.shape)
pre = np.argmax(predictions.predictions, axis = 1)

(22974,)
{'test_loss': 0.5201207995414734, 'test_runtime': 131.394, 'test_samples_per_second': 174.848, 'test_steps_per_second': 10.929}
(22974, 3)


In [18]:

correct =(pre == predictions.label_ids).sum().item()/len(pre)
correct                 

0.8159223470009576

预测准确率：0.8159
eval_loss: 0.5201207995414734
'loss': 0.3142,
'learning_rate': 2.2122670206293903e-07,
'epoch': 2.99


# 不使用trainer，自己多卡训练

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
BATCH_SIZE = 64
# 这里不用map，因为map只对一个batch做pad,要用tensordataset的话需要对所有数据做pad
# train_datasets = load_dataset("json", data_files=["./GeoEAG_data/train.json"])['train']
# test_datasets = load_dataset("json", data_files=["./GeoEAG_data/val.json"])['train']

train_encodings = tokenizer([[example["sentence1"], example["sentence2"]] for example in iter(train_datasets)], truncation=True, padding=True)
train_encodings['label'] = train_datasets['label']

test_encodings = tokenizer([[example["sentence1"], example["sentence2"]] for example in iter(test_datasets)], truncation=True, padding=True)
test_encodings['label'] = test_datasets['label']
print(np.array(train_encodings['label']).shape)


(96423,)


In [ ]:
def map_labels(example):
    example['label'] = label2id[example['label']]
    return example


train_encodings['label'] = [label2id[data] for data in train_encodings['label']]
test_encodings['label'] = [label2id[data] for data in test_encodings['label']]

tensor_train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']),
                torch.tensor(train_encodings['token_type_ids']), torch.tensor(train_encodings['label'] ).unsqueeze(1))
tensor_test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']),
                torch.tensor(test_encodings['token_type_ids']), torch.tensor(test_encodings['label'] ).unsqueeze(1))
# # np.array(dataset_all['train']['label']).shape
# # torch.tensor(dataset_all['train']['label']).unsqueeze(1).shape
# # torch.tensor(train_datasets['attention_mask'])
# # [[example["sentence1"], example["sentence2"]] for example in iter(train_datasets)]
# #max_len = len(train_encodings['input_ids'][0])
# test_encodings['label']

In [ ]:

trainloader = DataLoader(tensor_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validloader = DataLoader(tensor_test_dataset, batch_size=BATCH_SIZE, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 初始化BERT模型
model = BertForSequenceClassification.from_pretrained('bert-base-chinese',num_labels = len(label2id))#好像这里可以直接给label编码不需要自己重新做一遍，后面验证一下

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# for i,batch in enumerate(trainloader):
#     print(batch)
#     break

In [ ]:


model = nn.DataParallel(model)#多卡训练
model = model.to(device)
# 定义优化器和学习率
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
running_loss = 0
# 训练模型
EPOCHS =5
for epoch in range(EPOCHS):
    print('Epoch:', epoch + 1)
    for i,batch in enumerate(trainloader):

   
        
        batch_input_ids = batch[0].to(device)
        batch_attention_mask = batch[1].to(device)
        token_type_ids = batch[2].to(device)
        batch_labels = batch[3].to(device)
        

        # 计算模型的损失
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask,token_type_ids = token_type_ids, labels=batch_labels)
        loss = outputs[0].mean(dim=0)# 多GPU训练需要加.mean(dim=0)，否则直接使用outputs即可

        # 反向传播和优化
        loss.backward()
      
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
     
        if i%10 == 9:
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 10.0))
            running_loss = 0.0


Epoch: 1
[1,    10] loss: 0.93901
[1,    20] loss: 0.66309
[1,    30] loss: 0.62439
[1,    40] loss: 0.62227
[1,    50] loss: 0.57290
[1,    60] loss: 0.57498
[1,    70] loss: 0.58685
[1,    80] loss: 0.56125
[1,    90] loss: 0.58406
[1,   100] loss: 0.60031
[1,   110] loss: 0.59548
[1,   120] loss: 0.55234
[1,   130] loss: 0.52638
[1,   140] loss: 0.53376
[1,   150] loss: 0.52500
[1,   160] loss: 0.48453
[1,   170] loss: 0.49316
[1,   180] loss: 0.49524
[1,   190] loss: 0.56397
[1,   200] loss: 0.51885
[1,   210] loss: 0.50809
[1,   220] loss: 0.53007
[1,   230] loss: 0.54023
[1,   240] loss: 0.51940
[1,   250] loss: 0.50781
[1,   260] loss: 0.47171
[1,   270] loss: 0.47337
[1,   280] loss: 0.46466
[1,   290] loss: 0.46770
[1,   300] loss: 0.52230
[1,   310] loss: 0.50878
[1,   320] loss: 0.48410
[1,   330] loss: 0.48542
[1,   340] loss: 0.55199
[1,   350] loss: 0.47378
[1,   360] loss: 0.50344
[1,   370] loss: 0.48833
[1,   380] loss: 0.43848
[1,   390] loss: 0.46680
[1,   400] loss:

In [ ]:
with torch.no_grad():
    for i,batch in enumerate(validloader):

        batch_input_ids = batch[0].to(device)
        batch_attention_mask = batch[1].to(device)
        token_type_ids = batch[2].to(device)
        batch_labels = batch[3].to(device)
        
        # 计算模型的损失
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask,token_type_ids = token_type_ids, labels=batch_labels)
        loss = outputs[0].mean(dim=0)# 多GPU训练需要加.mean(dim=0)，否则直接使用outputs即可
        break


In [ ]:
outputs[0]

tensor([1.0522, 0.6858], device='cuda:0')